# Cactus reduction interface

Here you will find a step by step guide to create and test a Cactus thorn that uses the Cactus reduction interface to compute the center of mass of a distribution of matter on the grid. 

This notebook is geared towards users who are already familiar with the introductiory tutorials, in particular https://github.com/nds-org/jupyter-et/blob/master/CactusTutorial.ipynb  and https://github.com/nds-org/jupyter-et/blob/master/CreatingANewThorn-WaveEqn.ipynb .

If you find something that does not work, please feel free to mail users@einsteintoolkit.org.

## Prerequisites
Please make sure that you have followed the **Prerequisites** steps of the basic Einstein Toolkit Tutotiral notebook https://github.com/nds-org/jupyter-et/blob/master/CactusTutorial.ipynb .

## Notebook setup
This notebook is intended to be used online on the Einstein Toolkit tutorial server, offline as a read-only document, as a jupyter notebook that you can download and also in your own docker container using `ndslabs/jupyter-et`. To make all of these work some setting need to be tweaked, which we do in the next cell.

In [ ]:
# this allows you to use "cd" in cells to change directories instead of requiring "%cd"
%automagic on
# override IPython's default %%bash to not buffer all output
from IPython.core.magic import register_cell_magic
@register_cell_magic
def bash(line, cell): get_ipython().system(cell)
# Some versions of OpenMPI prevent oversubscribing cpus, which may happen if simfactory's
# number of cores detection is imperfect.
# OpenMPI by default pins MPI ranks to specific cores, which causes issues on shared
# system like the tutorial cluster.
# OpenMPI contains a bug affecting MPI calls with large amounts of data on slow systems,
# which can lead to hangs (OpenMPI issue 6568).
import os
os.environ["OMPI_MCA_rmaps_base_oversubscribe"] = "true"
os.environ["OMPI_MCA_hwloc_base_binding_policy"] = "none"
os.environ["OMPI_MCA_btl_vader_single_copy_mechanism"] = "none"

## Download

A script called GetComponents is used to fetch the components of the Einstein Toolkit. GetComponents serves as convenient wrapper around lower level tools like git and svn to download the codes that make up the Einstein toolkit from their individual repositories. You may download and make it executable as follows:

<b>Note:</b> By default, the cells in this notebook are Python commands. However, cells that start with <code>%%bash</code> are executed in a bash shell. If you wish to run these commands outside the notebook and in a bash shell, cut and paste only the part after the initial <code>%%bash</code>. 

In [ ]:
cd ~/

In [ ]:
%%bash
curl -kLO https://raw.githubusercontent.com/gridaphobe/CRL/ET_2020_05/GetComponents
chmod a+x GetComponents

GetComponents accepts a thorn list as an argument. To check out the needed components:

In [ ]:
%%bash
./GetComponents --parallel https://bitbucket.org/einsteintoolkit/manifest/raw/ET_2020_05/einsteintoolkit.th

In [ ]:
cd ~/Cactus

<h2>Configure and build</h2>

The recommended way to compile the Einstein Toolkit is to use the Simulation Factory ("SimFactory").
<h3>Configuring SimFactory for your machine</h3>

The ET depends on various libraries, and needs to interact with machine-specific queueing systems and MPI implementations. As such, it needs to be configured for a given machine. For this, it uses SimFactory. Generally, configuring SimFactory means providing an optionlist, for specifying library locations and build options, a submit script for using the batch queueing system, and a runscript, for specifying how Cactus should be run, e.g. which mpirun command to use.

In [ ]:
%%bash
./simfactory/bin/sim setup-silent

After this step is complete you will find your machine's default setup under ./simfactory/mdb/machines/&lt;hostname &gt;.ini
You can edit some of these settings freely, such as "description", "basedir" etc. Some entry edits could result in simulation start-up warnings and/or errors such as "ppn" (processor-per-node meaning number of cores on your machine), "num-threads" (number of threads per core) so such edits must be done with some care.
### Issues with GCC version 10
`gfortran` and `gcc` version 10 fail to compile the Toolkit due to stricter enforcement of language standards. To restore the old behaviour please edit the optionlist file ./simfactory/mdb/optionlists/&lt;optionlist&gt;.cfg mentioned in the machine defintion file ./simfactory/mdb/machines/&lt;hostname &gt;.ini by adding `-fcommon` to `CFLAGS`, `CXXFLAGS` and `LDFLAGS` and `-fallow-argument-mismatch` to `F90FLAGS`.

# The reduction interface

Cactus provides functionality to compute "reductions" of the value on the grid,
similar to the reduce operations defined by MPI. These operations include
finding the maximum or minimum value of a grid variable, or computing sums or
norms of grid variable data.

The exact set of operations is not defined by Cactus but instead determined by a
combination which driver thorn (PUGH or Carpet) and reduction thorn (PUGHReduce 
and LocalReduce or CarpetReduce) are active. Among the moe commonly used 
reductions provided by both drivers are:

<table>
    <tr><th>name (PUGH / Carpet)</th><th>description</th></tr>
    <tr><td><code>min</code> / <code>minimum</code></td><td>minimum value on the grid</td></tr>
    <tr><td><code>max</code> / <code>maximum</code></td><td>maximum value on the grid</td></tr>
    <tr><td><code>norm_inf</code></td><td>infinity norm the grid, i.e. $\max |\psi|$</td></tr>
    <tr><td><code>sum</code></td><td>volume weighted sum on the grid, i.e. Riemann sum</td></tr>
    <tr><td><code>norm1</code></td><td>volume weighted $L_1$ norm on the grid</td></tr>
    <tr><td><code>norm2</code></td><td>volume weighted $L_2$ norm on the grid</td></tr>
    <tr><td><code>average</code></td><td>volume weighted average on the grid</td></tr>
</table>

Note that Cactus does not define equivalents of MPI's `min_loc` and `max_loc`
operations.

The volume weighted sum is defined as
$\texttt{sum}(\rho_i; t) \equiv \left(\sum_i \rho_i(t) \, \Delta V_i\right) / \Delta V$ where 
$i$ runs over all the grid points, $\Delta_i$ is the volume of cell $i$ that
contributes to the sum and $\Delta V$ is the volume of a reference cell on the
coarset refinement level. For mesh refined simulations $\Delta V_i$ depends on
whether a given cell is (partially) overlapped by a cell on a finer refinement
level.

Cactus defines `sum` reductions such that

$$
\texttt{sum}(\rho_i; t) \, \Delta V \rightarrow \int \rho(t, x) \, d^3x
$$

as resolution is increased, where $V$ is the (constant) simulation domain.

## Using reductions

Reduction can be used either during output as options in the
`IOBasic::reductions` parameters, or at runtime in the `CCTK_ReduceGridArays`
(PUGH) or `CCTK_Reduce` (Carpet) calls.

When used for ouptut the output thorns will compute the requested reductions for
all output variables and write the result to disk / screen computing scalar
quantities.

The programmatic functions `CCTK_ReduceGridArays` and `CCTK_Reduce` allow
application thorns to do the same for example to write custom output files or to
store reduction output in grid scalars for later use.

In this tutorial we will focus only on the `CCTK_Reduce` interface offered by
Carpet.

```C
int CCTK_ReductionHandle(const char *reduction_name);

int CCTK_Reduce(const cGH *GH,          /* Cactus grid hierarchy */
                int proc,               /* MPI rank to receive result, -1 for all */
                int operation_handle,   /* obtained from CCTK_ReductionHandle */
                int num_out_vals,       /* number of output values per grid array */
                int type_out_vals,      /* CCTK_VARIABLE_INT or CCTK_VARIABLE_REAL */
                void *out_vals,         /* point to array of size num_out_vals for results */
                int num_in_fields,      /* speciﬁes the number of input ﬁelds */
                ...);                   /* variable indices of grid variables to act on */
```

## Example use

This fragments shows how to compute the `sum` reduction of `GRHydro::dens` which
is, up to the normalization factor $\Delta V$ the rest mass
$M_0(t) = \int \rho_*(t,x) \, \sqrt{\gamma(t,x)} \, d^3x$.


### Code
```C
const int sum_handle = CCTK_ReductionHandle("sum");
if(sum_handle < 0)
  CCTK_VERROR("Could not obtain 'sum' reduction handle: %d", sum_handle);

const int dens_index = CCTK_VarIndex("GRHydro::dens");
if(dens_index < 0) {
  CCTK_VERROR("Could not obtain variable index for GRHydro::dens: %d",
              dens_index);
}

CCTK_REAL rest_mass = 42.;
const int ierr = CCTK_Reduce(cctkGH, -1 /* proc */, sum_handle, 1 /* num_out_vals */,
                             CCTK_VARIABLE_REAL, &rest_mass, 1 /* num_in_fields */,
                             dens_index);
if(ierr < 0)
  CCTK_VERROR("Could not compute 'sum' reduction for GRHydro::dens: %d", ierr);

CCTK_VINFO("sum(GRHydro::rho) = %g\n", rest_mass);
```

### Schedule

```Perl
schedule restmass_computerestmass IN CCTK_ANALYSIS
{
  LANG: C
  OPTIONS: global
} "compute rest mass"
```

## Computing the to be reduced-grid function values

```Perl
schedule restmass_rho IN CCTK_POSTSTEP
{
  LANG: C
  OPTIONS: local
} "compute density"
```

This requires that:

* the grid function of interest must be computed in each timestep
* sufficiently many (three usually) timelevels are avaiable for interpolation in time
* the grid function must be checkpointed

otherwise the naive method of computing a grid function just before calling
`CCTK_Reduce` tends to either fail with an error message from Carpet or produces
subtly incorrect numerical values.

## Mesh refinement

Mesh refinement, in particular subcylcing in time, adds complications to
computing reductions (and any other "global" quantity.).

* a (`LOCAL`) scheduled function is called once per grid patch
* different refinement levels exist at different times
* `sum` like reductions have to take relative size of grid cells into account

<!--
#FIG 3.2  Produced by xfig version 3.2.7b
Landscape
Center
Metric
A4
150.00
Single
-2
1200 2
6 270 1073 1095 1350
6 270 1073 450 1343
4 0 0 49 -1 32 18 0.0000 4 195 165 270 1343 S\001
-6
4 0 0 49 -1 0 15 0.0000 4 210 660 435 1305 (t=4.5)\001
-6
2 1 0 2 0 7 50 -1 -1 0.000 0 0 -1 0 0 2
	 0 2340 6750 2340
2 1 0 2 0 7 50 -1 -1 0.000 0 0 -1 0 0 2
	 0 4500 6750 4500
2 1 0 2 1 7 50 -1 -1 0.000 0 0 -1 0 0 2
	 1350 3420 5400 3420
2 1 0 2 1 7 50 -1 -1 0.000 0 0 -1 0 0 2
	 1350 2160 5400 2160
2 1 0 2 1 7 50 -1 -1 0.000 0 0 -1 0 0 2
	 1350 1260 5400 1260
2 1 0 2 0 7 50 -1 -1 0.000 0 0 -1 0 0 2
	 0 270 6750 270
2 2 0 2 0 7 50 -1 -1 0.000 0 0 -1 0 0 5
	 0 0 6750 0 6750 4500 0 4500 0 0
3 2 0 2 0 7 50 -1 -1 0.000 0 1 0 3
	1 1 1.00 60.00 120.00
	 810 4500 1350 2880 900 1440
	 0.000 -0.500 0.000
3 0 0 2 0 7 50 -1 -1 0.000 0 1 0 3
	1 1 1.00 60.00 120.00
	 810 270 360 630 540 1080
	 0.000 1.000 0.000
3 2 0 2 1 7 50 -1 -1 0.000 0 1 0 3
	1 1 1.00 60.00 120.00
	 2610 1260 1800 630 990 1080
	 0.000 -0.500 0.000
3 0 0 2 0 7 50 -1 -1 0.000 0 1 0 3
	1 1 1.00 60.00 120.00
	 810 2340 270 1890 540 1440
	 0.000 1.000 0.000
4 0 1 50 -1 0 15 0.0000 4 225 810 4680 3330 rho_p_p\001
4 0 1 50 -1 0 15 0.0000 4 165 510 1440 3330 t=3.5\001
4 0 0 50 -1 0 15 0.0000 4 165 510 90 4410 t=3.0\001
4 0 0 50 -1 0 15 0.0000 4 165 510 90 2250 t=4.0\001
4 0 1 50 -1 0 15 0.0000 4 165 510 1440 2070 t=4.0\001
4 0 1 50 -1 0 15 0.0000 4 165 510 1440 1170 t=4.5\001
4 0 1 50 -1 0 15 0.0000 4 165 330 4680 1170 rho\001
4 0 1 50 -1 0 15 0.0000 4 225 570 4680 2070 rho_p\001
4 0 0 50 -1 0 15 0.0000 4 165 510 90 180 t=5.0\001
4 0 0 50 -1 0 15 0.0000 4 225 810 5760 4410 rho_p_p\001
4 0 0 50 -1 0 15 0.0000 4 225 570 5760 2250 rho_p\001
4 0 0 50 -1 0 15 0.0000 4 165 330 5760 180 rho\001
4 0 0 49 -1 12 15 0.0000 4 195 2100 3150 720 sum(rho;t=4.5)\001
-->

<img src='data:image/png;base64,
iVBORw0KGgoAAAANSUhEUgAAAsoAAAHeBAMAAABwFzBjAAAAMFBMVEUAAABxcXGqqv84OP8lJSUbGxtUVP8lJf+qqqoSEv9UVFT///84ODgSEhIAAP8bG/+ATo1xAAAW10lEQVR4nO2dv47jypWHmRhoSDbgZ5hASQP9AgvsM2ziyI7GAJMG9AwKB5KfYcGwQSUC7eUS5IX25ptssi8wF1DSADdzvFUs/ilJ9b+KlJr8fZi+w6Fuq9Wfjs45VSSLUQXGJ3r0C1gEURmBsYHlKSCWV4/+OM0eWJ4CoeU0TZu/9+nH5C/oa5Aerf53kWWSqJtdxUuVHEK9rllRJq6Wi37fqVV7OtI/QICz5X2/r9siT4WcLcbVcjH47LIxGa/AshhXy7tV2u4q2rTcjAoxMryjPCaH5JhQR/vIrD3oLQ9RW6RJRN+qEpZFlMkqWSdpSvPp0bA94C0nzTCFCC6SlwqWZexeigMRdFpTxacXk2/hLe9TyqFqAxuWxRC9NC+Tv4wLlyBjsN2wLOPKcmEkSGI5qdongOU7esvmYSjOGI1l87dqWfSWzcNQWP1Iy0z/Q5I7+uV7uIxBItInL79UxaHakV5lPcbr/NoMlmkzZ9djFFHRjmdOH1VJFK+p4h3mMe7Y9ZaJ4tJI0DCPkXTBfIroEJs+V5oiYdxBRiVH8vUjWR2qJE3131CJZz73/XjG7DmADhwrmQJYngJYngJYngJYngJYngJYngJq+XcpGBecJzcJNrH8azTmGz5jbPJyGWHuyA0bywnKpCMWlhHKzphbLiKjCWsgwNzyLsL5n64YW0Yoe2BsGaHsgalllD4fTC2ji/PB0DJC2QtDywhlL8wslyh9XhhZZuczA2eMLKOL88TEMgYkvphYRij7YmAZoeyNgWWEsjd6ywhlf/SWEcr+aC0XGFv7o7W8w/U7/mgtIysHQGcZMxgh0FnGDEYINJYRykHQWEYoB0FtGW1cGNSWdzhEEgS15QQXCAdBaRm1LxBKyyckjDAoLX+NhDGse3Aymw3gf62Enig/+qXQKsvF10gY3LoHJ9Hjt5f9F2vOcqt35DUgVZa/yMVTybBpZLk83lseeR0QlWWzVbwUZPG2ruP41fNpNHAv38hyWt1bLsYNKJVly2UWb8niuiX2ep6OMorSQxWtSfo9llGyas8qa8IwicifQ3Uqo9Wh+eeqX7DtRK9R4lPfy5Xlkq1u+DjLfgO/DdUbv+c0ni/vPs/UQhST932/puvyFskHedlNIDaWy2T/QTZOafNPYm5PP4fNGo8FvUipqtpVmZrQvq5+zSc2EfzAcCgt+zwxzRVtqsjIdgDNa1ak2F8J1dlYZoWLNZ10e8XSAP1iB9PajLGntukiNhVveX9gi0h6fm41KCyXHpaJWD5+NyE0r9jHvrd8vLLcPEjz8ormBLa7aLLBTV5Oryx3j4+7QJPKsnuuopKvduQBNO+jFVUhsVx1/121OaTX1m6x6xubf91YXldjt3IKy+4jPyL5+82uEJqrPf1w+1qm2flKKSufD7Ps2kMKJFPNF7+OrjhUXSBqLBddcJasPN4JFGSMcW8AMIJloeRGs9vztVAZNOzWTRIdLLMBBW+52U+bBnZuu8Tyrln9+NC+TQ/rMVwtiyXTEvjT7QkZ9GUmTQ9BG+XBctF0YleWd+2ahexID9ksrucpml8sidiTssce1i87WpZJpo/4pOYyZSvJ76LVLiojumIsm/ahO1myPUV0sdJ1VbSjkjb/kp6YX1i9ZHmZBfqe/aMY98hbcMsbeWLItn6puR2m3c6fiT7szcq7/YK7qSjptgtusnxitpCkM6EtK7Nv7pczJEgNWbSiu3FnHwNb1nQSbyHGgLcUMssWv8Dj5pcdLGe61OuZM+wonmZ6PKxlbX3LRskZT4/KsnXd3Ujbi55RcsbTE3Iew2jcsfUbm3xNAlo2a9TG6TOenICWDQcdb/WEBfBJUFi2vIuGQVJuyBaYM5THSmw6IfPJoHx5BTDU0VWb0fPygjmUZZuZoMwwt8wHpWXzYYlpUmYsrgAqz3oxtpzZzdAvbgSotGzcym0tg3OzsAIY5Dw5+2Hzwgqg0rJhw+wwnFtYN6c8f9msYXYaZiwrmNVniRu1ck6H85ZVAP0t527d76K6OaXlnUErZ9nEcd+3oGBWX73zoj8eZtvE9WwWFMxKy79E2i7DMV9QFlQAVZb/Fmktu+YLyoK6OeU8hsZy7pEvKMsJZpXlXzSW3/7FI19USwpm9fV+kWpcktW/eZ7GuZhgVvfLkeoCnnNde7YJ2VKCWddjKCz/a11/ev70pQxNlJYLleW/k1D+zfOnL2VoolkfI/qz9LE3er2k+hPfXk2Zx7Hs/3twMOdhrvfUorb8SyQdY2d6yVlb3BSX+z04mOOJyq9u3SKp5f+USM6Gzbz9HVQiHxvM+VNYrv4otfzv9W9CP5th8zv7HTJVxGfbRwbzk1j+RfbYf0kkZ8Przt/Zdq5cVCB/ZDA/iWXZY1n9f+IH3i59QdlU7Hd4U+fvhwVznP/ML8340+48Bwc0lou9eH8mi08uOt5by5tYOUR8VDDHdVznl5gUwPy92o77Glzvh/1Nsp9Y3jZrYrznr1UnVz2V/KBxdla/fqPvcE0Vn8f9QIW+6zixvIkpr6QM9vq2qpTxqEmjmn3y6m5jREaw3PLJWT4rh+IPCmbeclaP+qNGs0wGfHFfCXOl5QcFc2c5r9k/RmS0jLFpsnO3Vz2t9Jhg7ixnX9JyW/0qLmPk6mB9TDAPGeOVe6mjMF5erpqXnjXdxVnTKD0kmHvLpDbnj+wxruc92wueyzQ93jwykNXcgJr8Dm+f1eW10jX9jwjmrLdMFGs+bL4ILDcrgjWLJNwMSXb9QgjH24cG+LBkluuLft2+BwQzGZWQr+903LeNR54BFVhu7lCyX1XNyjQF90B7QldzSav0EvN7MoPh68yPTQksszUHE5YhuJgtWSy3fgNfhf826wOtAsvs9lynRjS//mXK1mEq2Zoep7AX7M87mEXVr1lrpmQrAK36mC2OzPKOHQwMvQDbrINZZPnULlO6upr5LFux+7SpjKHX0p11MIsst8ehWstsFdJj9dGH756eCxN8ceY5B7OwX27XYVs3LptVSNMDKXp9kqDNRnDLcw5moWW2vvxpzbs8can4+pFQzDiYhZbZnmuXJJ6TtNVcCi3Hnvy1/qfvUwQn0JEqseUjZ7nNGE16Xqss1zMk0MdLaJmtrnZT/biMUQrzsnfg/DVA8AVmxFhul2O86eSY5R/NxqFiC2kCM0SWu8UG6VRGxE1YEMtFRJf0pitBjnzviXkhssyGe3tucV0GsxytmiH2yPdRmReieQyWCz6axYnvKcRrmgIFAsunpqlI1urZTaRlCwSWWVMRHe9m8XnEcQ7E3FsumWQ6VyEPZotJfBD+6CoQActTAMtTAMtTAMtTAMtTAMtTAMtTAMtTAMtToLtGSjD1Nu7tw2aJw1WV495wcJborhAWTHAm/z3ey5kpuqvdBUZPfxjx9cwTteWT0LLVXQlApbP8q2gNrn9EOE5iicPaiGX0+xFf0CxRWv5V6LOIRHkEKFCvJR4JH5LsBlJUlneROAOrF/MD9yjvIxWJly36N8l+IEN1F0Vi+c+iB5II9c8O1Z3nIsmifQlShiWqvPwfEss75cKU4B5l9fu9eGHg0x/s7vwH1PdEK4X7ywNi2Q5ljyFNvhZ3SwOV631X7W/ivGxUnZzcMg5i2aGwrEgLlje+XTwqy4oSh6NSVsgtK4ofyp8lcsvK3Gt+G0tQqSwr+wiUPyvklpVJQZlOwC1yy+rxHcqfDVLLmmbN5NZ/oENqWZN5Uf5skFrWaDS9vzCgSC3rUgLKnwVSy7ryhvJngdSybgoZ5c8CuWXNN6L8WSCzrK1uKH8WSC1rQxXlzxx3yyh/5sgs6485ofyZI7WsdYjyZ467ZZQ/c9wto/yZ42EZ5c8YD8s7nJRhiodlHJUyxscyyp8p7v2yfkIJdLiP/VD+zPGxjPJnisyyyZlwKH+myOeX9WMOlD9TnI9INd+MxGyG89FVCsqfIa5nCjToyl9717xNqEX8g5OPfF+/DvlZLyaJWf1OZOw+FNnPsW+d7kw80Y1oXM+TaxCVv2zYbO/Ben6nf56SkW823iO3bNINC+J9M2x+Z78DvXnsk9686PGWTfq0+/KXDa87f2fb49+g3pnHWy4MUsZ9I/LW38+dRHVveeSbersR5z/zS3P3YnqH25b8fYwb7aqu3tGnjPtGhIuOd2Z5ghvUOxHXcZ1fYlIA8/e+PGfbSzzCHfCUawpov/v+fxnu7Z6/PrflrH79ltev9K7jr9W5u+v428/qHD6YVddhG6SMu/JHLG+a+1y9bqrntjzc250vHOdP9oLDolxTQD8wuSt/Q8b4/EqWs+4FEssjvFj16lDaYL4rf73lnMRzUwmzZ7d8FQbTWzaof3flr88YmyY70118qDwTneXswZb1o+y78jdUv6rNGLS4PGW/PGSM1+6lMsvTVj+TYL59H658ku3stXojY79P19c3Ir1lOjjtegxa/X4qv80F3WqqumC+LX9ZnQ3tJvkd3j6p4rcnnMfIestEa969wDO3HQ7dysC6YL4rf/zQiVmu4qmmvqwgoxLy9Z2O+7ZxP2A9X8Z4sZpVrne6YDY683OiWdwQnEfJbbp18XXN3NyOsE5pub2/7T790I1MZnPZ+7npjD4NLaf6YTGP2PKueZLipUp+6IJ5Zsf+3owsl5a/9WC54PayJzkdyR9dMM9r1Zd8ezHqMJwtczdZLVksk6cqV7rMvMxFz1wtF1zMpuxm11FT3DTBPLfyZ4ar5d2qv81Ocewtky9NMC/vBKPymBySY3M79n300e1NVXNrvWUuJkt24/aSWdYFc//k9QwRjFDKZJWskzSl+fRYJW2LtY/SRO6Jt5y098H+uLasGQAmC7Nc7V6KQ/JCz6MniruyRDtaeTDzlvfNTd0PxfHasmYAuLirK+llCiS0yF8RlwIiVbslyBin6sayOpgXd3XlleX+DORIdZWIwDKJ54SObYrOsjqYF9dk9Jb7MKwqC8ttxmjS87qqhrdKGcyLW1m1t1w4We6qX5sxaHJvA1UZzDMbY2vhMgaxwuXlk1H1457oWBUHOptRsrenUB00mdcYW89gmTZznZio6ru6e4axX1QMbwWxTFoHonjX7lMdAVzaGHvXWyaKy04aEZRKv2WYx+CbamaZ1MF+n6LlXlj5I6OSI/n6kawOFRmcdLujVJE51bP4w7ujGD8ub4wtQinB+E7NqvHjTCbyvQhjWRHMS2syrjk1ndk6jGXFpNHSmgwR6lGDuWX50GRpTYaINJJ3GDaW5UOThTUZDlhYlgYzmgwdFpblwYwmQ4ON5ULWzS27yTCAWo7A2MDyFFhkDCloMjSEsYwmQ00Qy2gyNISxjCZDDSxPQRjLizsnw5Iwlhd3ToYlYSyjlVMTxvLizsmwJIxltHJqAllGk6EklGU0GSoCWTZYFHvJhLKMJkNFIMto5ZSEsoxWTkUgy9rbAS6bYJbRyikIZRkNs4pgltHKKQhlGQ2zimCW0copCGUZDbOKYJbRyikIZRkNs4pwljHDLCeYZTTMCmB5CsJZRsMsJ5hlDEsUwPIUhLOMYYmcYJYx+FMAy1MQzjIGf3KCWcYQWwEsT0FAy5jIkBLOMobYcmB5CmB5CmB5CgJaxkSGlHCWcZ2UnHCWMSknB5anAJanAJanAJanAJanAJanAJanAJanAJanAJanAJanAJanAJanAJanAJanALP4U4DjflMAy1MAy1MAy1MAy1MQ8GxEWJaCcz6nAOcvTwEsTwGuK5kCXCM1BbA8Bbh2dQpwHfYUBLOMiU8FWIVkCmB5CsKtDgXLcrDS2RSEsow7lqjACpRTAMtTEGzNWtzfXQHWX54CrCU+BVgXfwpGvMdDXL+zjTyO3wP8iBHI42l+znj3K8nj7YVtbev6SS3Hl2l+TvB772Tdxreqql+brZ/+zz8S+Rez3D/Hptsg4bttQjh70kCuvpzl/khJxr/ubfPfPH71/wHj8MUs93P4bxeuoLBU8fakaTnOf+aXnL62Tf2925m/by+ibT9C3xGUj46cyd3E9UQhY0Vcx3V+iUkBJK9z237csu2lb434bU9GsExaipqGb97L3dRPmDOy+vVbTl5YTRWfuwr99rM6XwTbfoS+hzCxvIkpr1zD3JbBJ6Nmn7y622g4f5LoEGz7Efp+2Ff15O2z3Th/Vs8HbznjzFaibT9C39v9ynL+ebvxTHSWm3j9CpZPvOU+Y3wRy9lXsTycjMFVP97yc+dlEhFcXhZu+xHG8jAjd50x3qus6S7OT9hjDJZJbc67HoNWPNG2H6EtZ3U3oCY5Y9MUwMtr9V32jQ8k6y0Tlf2H7SzZ9iOMZe5qh364RHr698ZyfXnKITYZlZCv73Tct437Aev5MszT8dt+BLEsPEM861559oyRLIPvOcP1n2Esz+cA9hNbnsMB7HPTGX02ZvntIASxPKNDq2+f4m0/wliezaHVfHt5F217EsQyLqjUEMIyLkLTEcTyfFqMkQhhuW0x4hkSqNUPYbltMeoZ8kRjv7bFeHTcjcETxTJaDB0BLKPF0BLCMloMHQEsz2EWY2QCWJ7RLMZYhLA8m1mM0QhgGS2GFn/LaDH0BLCMFkOLv2W0GHr8LaPF0ONvGUs26PG2jMUEDPC2jOJngLdlrHBmgLdlrL1lgLdljK8N8LaMFsMAX8sYX5vgbRnFzwBfyxhfm+BrGeNrE3wto/iZ4GkZ42sjPC2j+BnhaRnjayM8LaP4GeFrGcXPBE/LOH5thJ9ljK/N8LOM4meGn2VMLpvhZxnFzww/yyh+ZnhZRvEzxMsyip8hXpZR/AzxsoziZ4iXZUwuG+JjGcXPFC/LKH6G+Fje4ciqIT6WUfxM8bGM4meKh2UcWTXGwzKKnzEelnFakTEelnFk1Rgfyyh+prhbLjC5bIy7ZRQ/c9wtY3LZHHfLKH7meFhG8TPG2TKKnwXOllH8LHC2jJGfBc6WUfwscLaMaU8LXC1j2tMGV8sofja4Wkbxs8HVMoqfDa6WUfxscLSM4meFo2UUPyscLaP4WeFoGcXPCkfLKH5WuFlG8bPDzTKKnx1ulnG2px1ulnE0yg43yyh+djhZxqUOlrhZRvGzw8kyip8lTpZR/CxxsoziZ4mLZRQ/W5wso/hZ4mIZxc8WF8sofra4WEbxs8XBMoqfNQ6WcRK+NSLLZdLu2acfgm/BCi9VldolTZHlNGWZt3ipEsGp4Ch+JA5dLRfdruLQHqI+Hemf+2/BSfi2kTZY3ne7ym7cQZ5KkLNR/Cp3ywXvs1yzx0SWUfwqd8u7VTrsPR3YY+zr5gcsvfiVx+SQHJsOYR/17UFaDsMIfruht3wVtY3wUmJ54cWPdGDJOklTmk+PfXuwj9KuM7vaZvCWk4hCJCbNWyG2jCvQdi/FIXmpTmuquDuVjWrpApjfZvCW9ynl0DXMYsuYkCN66Qea/MUXrojLpdFtXhVnjObcIbFlFL8ry/1ZVhF74G673SGyXCVV+wR3lhdf/AbLV2FobLnPGMzy1VvVs/Tix1kunCz31a+1TJL7XRZG8eMzBnHB5eV+oMxvtzuEGSOl4+zdsR2ecKD4cZZpM9cdN4qqYdKH3253dGO/qGj1lx8V2dytqeLdbX7ASfjNnCSzTBSXffweqn5Yx2+zHf08Rt9Il1GasudK07vIRVombe6RfP1IVocqSQez6aCG32Y7BDOf+2GoePczcDRKQiTZbndYHSvBhJyMoJZR/G45NZ3ZujHbbEbelnEqhgR+YHE/yLC0jOInIY1S4TbD0jLGJE7YWUbxc8POMibk3KCWf/cnU/7H/H8FHNRyBMamigqLN+V/x3q3581fBMdPQXj+HxTGz8V612r8AAAAAElFTkSuQmCC
'>

## Computing the reductee on the fly (I/II)

### Mixing compute and reduction calls
Not wanting to store each reductee indefinitely on the grid, we want:

```C
CCTK_LOOP3_ALL(compute_dens, cctkGH, i,j,k) {
    int idx = CCTK_GFINDEX3D(cctkGH, i,j,k);
    dens[idx] = ...;
} CCTK_ENDLOOP3_ALL(compute_dens);

CCTK_Reduce(cctkGH, -1 , sum_handle, 1, CCTK_VARIABLE_REAL, &rest_mass, 1,
            dens_index);
CCTK_VINFO("sum(GRHydro::rho) = %g\n", rest_mass);
```

which is impossible to schedule since either (in `GLOBAL` mode) we have no access to the data on the grid or (in `LOCAL` mode) we cannot call `CCTK_Reduce` to compute a "global" quantity. It also does not set `dens_p` or `dens_p_p`.

### Mode changing macros

Carpet provides macros in its `carpet.hh` public header that allow user code to switch modes and iterate over all refinement levels and grid components:

* `BEGIN_REFLEVEL_LOOP(cctkGH)` ... `END_REFLEVEL_LOOP` to loop over refinement levels
* `BEGIN_LOCAL_MAP_LOOP(cctkGH, grouptype)` ... `END_LOCAL_MAP_LOOP` to loop of each coordinate map on the current level
* `BEGIN_LOCAL_COMPONENT_LOOP(cctkGH, grouptype)` ... `END_LOCAL_COMPONENT_LOOP` to loop over grid components
* and `DECLARE_CCTK_ARGUMENTS` to set the grid function pointers

`grouptype` is `CCTK_GF` for regular grid functions.

Carpet uses these same macros when traversing the schedule bins.

## Computing the reductee on the fly (II/II)

Using mode changing macros we schedule in `GLOBAL` mode and compute all data on all refinement levels, coordinate maps, and grid components as needed:

### Code
```C
#include "carpet.hh"

BEGIN_REFLEVEL_LOOP(cctkGH) {
  BEGIN_LOCAL_MAP_LOOP(cctkGH, CCTK_GF) {
    BEGIN_LOCAL_COMPONENT_LOOP(cctkGH, CCTK_GF) {
      DECLARE_CCTK_ARGUMENTS;
      CCTK_LOOP3_ALL(compute_dens, cctkGH, i,j,k) {
        int idx = CCTK_GFINDEX3D(cctkGH, i,j,k);
        temp_dens[idx] = ...;
        temp_dens_p[idx] = ...;
        temp_dens_p_p[idx] = ...;
      } CCTK_ENDLOOP3_ALL(compute_dens);
    } END_LOCAL_COMPONENT_LOOP;
  } END_LOCAL_MAP_LOOP;
} END_REFLEVEL_LOOP;

CCTK_Reduce(cctkGH, -1 , sum_handle, 1, CCTK_VARIABLE_REAL, &rest_mass, 1,
            temp_dens_index);
CCTK_VINFO("sum(GRHydro::rho) = %g\n", rest_mass);
```

### Schedule

```Perl
schedule restmass_computerestmass IN CCTK_ANALYSIS
{
  LANG: C
  OPTIONS: global
  STORAGE: temp_dens
} "compute rest mass"
```

# Putting things together

To demonstrate all concept we will write a new thorn to compute the center of mass (normalized mass dipole):

$$
M_i(t) = \int \rho(x^i, t) \, x^i \, d^3x, \\
M(t) = \int \rho(x^i, t) \, d^3x, \text{and} \\
C_i = M_i / M
$$

using the rest mass density `HydroBase::rho` for $\rho$. Note: the "mass" $M$ defined this way does no correspond any physical quantity since it is defined without using the metric determinant $\sqrt{\det g_{ij}}$.

## Skeleton thorn

Cactus comes with the phony make target `newthorn` to create a compilable but
empty thorn setting up a skeleton of files.

In a shell you would use

```
make newthorn
```

then respond to the prompts as directed by the script. In this notebook, we
automate responses to the script's prompts.

We then overwrite

* `configuration.ccl`, `interface.ccl`, `schedule.ccl`
* `src/compute.cc`, `src/make.code.defn`

with our own files.

In [ ]:
# prompts by the newthorn script and responses to provide
newthorn_chat = {
    # prompt: response
    "Thorn name": "CenterOfMass",
    "Pick one, or create a new one.": "EinsteinAnalysis",
    "Thorn Author Name": "Roland Haas",
    "Email Address": "rhaas@illinois.edu",
    "Add another author?": "n",
    "Licence": "GPL",
}

In [ ]:
# this drives the chat, logging everything to the notebook
import subprocess

# remove any existing thorn by that name, the chat hangs otherwise
print(subprocess.check_output("rm -rvf arrangements/EinsteinAnalysis/CenterOfMass/", 
                              shell=True, universal_newlines=True))

# "PERLIO=unix" ensures that prompts are visible and not hidden by buffering
newthorn = subprocess.Popen("make newthorn PERLIO=unix",
                            stdout=subprocess.PIPE, stdin=subprocess.PIPE,
                            text=True, shell=True, universal_newlines=True)

prompt = ""
while not "Creating thorn" in prompt:
    prompt = newthorn.stdout.readline()
    print(prompt, end='')
    for tag in newthorn_chat:
        if tag in prompt:
            print(newthorn_chat[tag])
            newthorn.stdin.write(newthorn_chat[tag] + "\n")
            newthorn.stdin.flush()
(stdoutdata, stderrdata) = newthorn.communicate()
if(stdoutdata): print(stdoutdata, end='')
if(stderrdata): print(stderrdata, end='')

## interface.ccl

The interface file defines grid functions and grid scalars provided by
our thorn, it also states the name of our thonr's implementation.

Since we need access to the coordinates we inherit from `Grid`, which is usually
implemented by thorn `CartGrid3D`, and `HydroBase` to have access to `rho`.

We also declare that we use Carpet's provided header files `carpet.hh` to gain
access to Carpet's internal macros and data structures.

In [ ]:
%%writefile arrangements/EinsteinAnalysis/CenterOfMass/interface.ccl
# Interface definition for thorn CenterOfMass
implements: CenterOfMass

# Grid provides coordinates x,y,z, HydroBase contains rho
inherits: Grid, HydroBase

# use include file provided by Carpet describing its internal functions
USES INCLUDE HEADER: carpet.hh

CCTK_REAL tmp_rho_xi TYPE=GF TIMELEVELS=3 tags='checkpoint="no"' "integrand for center of mass"

CCTK_REAL center_of_mass TYPE=SCALAR TIMELEVELS=1
{
    M, Cx, Cy, Cz
} "center of mass"

## schedule.ccl

The schedule file schedules both functions and storage for our thorn. 

In [ ]:
%%writefile arrangements/EinsteinAnalysis/CenterOfMass/schedule.ccl
# Schedule definitions for thorn CenterOfMass

STORAGE: center_of_mass
STORAGE: tmp_rho_xi[3]

# ensure that HydroBase::rho has the 3 timelevels I assume it has
STORAGE: HydroBase::rho[3]

schedule CenterOfMass_Compute IN CCTK_ANALYSIS
{
    LANG: C
    OPTIONS: GLOBAL
} "Compute center of mass over grid"

## configuration.ccl

Our thorn uses Carpet's interal macros so must be linked against `Carpet` as well as have `Carpet` active at runtime. 

In [ ]:
%%writefile arrangements/EinsteinAnalysis/CenterOfMass/configuration.ccl
# Configuration definitions for thorn CenterOfMass

REQUIRES Carpet

## src/centerofmass.cc

The scheduled function has to compute the three dipoles
$M_i = \int \, \rho \, x^i \, d^3x$ as well as a normalization factor (mass)
$M = \int \rho \, d^3x$ and then set the grid scalar to the result
$C_i = M_i / M$.

In [ ]:
%%writefile arrangements/EinsteinAnalysis/CenterOfMass/src/compute.cc
#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"

#include "carpet.hh"

extern "C"
void CenterOfMass_Compute(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_PARAMETERS;
    
  const int sum_handle = CCTK_ReductionHandle("sum");
  if(sum_handle < 0)
    CCTK_VERROR("Could not obtain 'sum' reduction handle: %d", sum_handle);

  const int tmp_rho_xi_vi = CCTK_VarIndex("CenterOfMass::tmp_rho_xi");
  if(tmp_rho_xi_vi < 0) {
    CCTK_VERROR("Could not obtain variable index for CenterOfMass:tmp_rho_xi: %d",
                tmp_rho_xi_vi);
  }

  // compute M, M_x, M_y, M_z as components of [M, M_x, M_y, M_z]
  for(int c = 0 ; c < 4 ; c++) {
    // compute integrand on whole grid for all past and current timelevels
    BEGIN_REFLEVEL_LOOP(cctkGH) {
      BEGIN_LOCAL_MAP_LOOP(cctkGH, CCTK_GF) {
        BEGIN_LOCAL_COMPONENT_LOOP(cctkGH, CCTK_GF) {
          DECLARE_CCTK_ARGUMENTS; // in local mode
          const CCTK_REAL *xi[4] = {NULL,x,y,z};

          #pragma omp parallel
          CCTK_LOOP3_ALL(CenterOfMass_Compute, cctkGH, i,j,k) {
            int idx = CCTK_GFINDEX3D(cctkGH, i,j,k);

            tmp_rho_xi[idx] = rho[idx] * (xi[c] ? xi[c][idx] : 1.0);
            tmp_rho_xi_p[idx] = rho_p[idx] * (xi[c] ? xi[c][idx] : 1.0);
            tmp_rho_xi_p_p[idx] = rho_p_p[idx] * (xi[c] ? xi[c][idx] : 1.0);

          } CCTK_ENDLOOP3_ALL(CenterOfMass_Compute);
        } END_LOCAL_COMPONENT_LOOP;
      } END_LOCAL_MAP_LOOP;
    } END_REFLEVEL_LOOP;

    // integrate
    DECLARE_CCTK_ARGUMENTS; // in global mode
    CCTK_REAL *Mxi[4] = {M, Cx, Cy, Cz}; // direction c's result
    const int ierr = CCTK_Reduce(cctkGH, -1, sum_handle, 1, CCTK_VARIABLE_REAL,
                                 Mxi[c], 1, tmp_rho_xi_vi);
    if(ierr)
      CCTK_VERROR("Could not compute integral of tmp_rho_xi: %d", ierr);
  }
    
  DECLARE_CCTK_ARGUMENTS; // in global mode
  *Cx /= *M;
  *Cy /= *M;
  *Cz /= *M;
  // "sum" only uses relative cells sizes, rescale by reference cell to get
  // intgral value
  *M *= cctk_delta_space[0] * cctk_delta_space[1] * cctk_delta_space[2];
}

## src/make.code.defn

Finally let Cactus' build system know which file to compile

In [ ]:
%%writefile arrangements/EinsteinAnalysis/CenterOfMass/src/make.code.defn
# Main make.code.defn file for thorn CenterOfMass

# Source files in this directory
SRCS = compute.cc

# Subdirectories containing source files
SUBDIRS = 

# Compiling the new thorn

In [ ]:
# Make sure the new thorn is in our ThornList
# outside of this notebook one would obviously use an editor

contents = None
with open("configs/sim/ThornList", 'r') as f:
    contents = f.read()
if "EinsteinAnalysis/CenterOfMass" not in contents:
    with open("configs/sim/ThornList", 'a') as f:
        f.write("EinsteinAnalysis/CenterOfMass" + '\n')
print(subprocess.check_output("tail configs/sim/ThornList", 
                              shell=True, universal_newlines=True))

In [ ]:
%%bash
# compile everything

time ./simfactory/bin/sim build -j2 sim

## Cactus test cases

Please consult the excellent
[Adding a test case](https://docs.einsteintoolkit.org/et-docs/Adding_a_test_case)
tutorial on the ET wiki by Ian Hinder for more details.

Cactus test cases are:

* regression tests
* small
* quick

Cactus test cases are not:

* convergence tests
* physics correctness tests

Verifying physical correctness is the author's duty when writing the code and
may be achieved by providing example parameter files and / or special "test" 
settings.

A test consists of:

* a parameter file
* expected output stored in files

Cactus compares the new and the expected output by comparing real numbers in
ASCII files, allowing a certain "fuzz", i.e. a difference of about 1.0e-12 by
default.

### How to design a test case

Test cases should be

* simple
* small
* quick
* use as few thorns as possible (simple!)
* output only a few variables (small!)
* output only norms and 1D ASCII quantities (small!)
* finish in under two minutes (quick!)

Preferably, each major feature in a thorn should be covered by a test case. If necessary, one has to introduce helper thorns that use the tested features.

## Test parameter file

We will use use a modified version of the TOV example paramter file in
[CactusTutorial.ipynb](https://github.com/nds-org/jupyter-et/blob/master/CactusTutorial.ipynb)
to compute the center of mass of an off-center neutron star.

Thus we will modify the initial star location to be

```Python
TOVSolver::TOV_Position_x[0] = 3.5
TOVSolver::TOV_Position_y[0] = 1.5
TOVSolver::TOV_Position_z[0] = 2.0
```

and adjust the initial grid setup to be centered on the same location

```Python
CarpetRegrid2::position_x_1 = 3.5
CarpetRegrid2::position_y_1 = 1.5
CarpetRegrid2::position_z_1 = 2.0
```

Since this breaks reflection symmetry we remove the `ReflectionSymmetry` thorn
and related parameter settings. While not strictly needed for the regression
test, having "recognizable" results makes it easier to track down test failures.

```Python
CoordBase::xmin = -24.0
CoordBase::ymin = -24.0
CoordBase::zmin = -24.0
CoordBase::xmax = +24.0
CoordBase::ymax = +24.0
CoordBase::zmax = +24.0
        
CoordBase::boundary_shiftout_x_lower    = 0
CoordBase::boundary_shiftout_y_lower    = 0
CoordBase::boundary_shiftout_z_lower    = 0
```

We add our new thorn

```Python
ActiveThorns = CenterOfMass
```

and request output for its grid scalar

```Python
# these makes "timeseries" output via out0d_vars readable
CarpetIOASCII::compact_format = "yes"
CarpetIOASCII::one_file_per_group = yes

CarpetIOASCII::out0d_every = 1
CarpetIOASCII::out0d_vars = "CenterOfMass::center_of_mass"
```

and remove all unneded thorns and output:

* CarpetInterp, CarpetIOBasic, CarpetIOScalar, CarpetIOHDF5
* ReflectionSymmetry
* AEILocalInterp, LocalReduce
* Formaline, NaNChecker, TerminationTrigger
* 1d-ASCII output

Finaly since this is a test we reduce runtime and disable some output that would
make introduce ever changing timestamps in ASCII file comments

```Python
MoL::ODE_Method             = "Euler"
MoL::MoL_Intermediate_Steps = 1
MoL::MoL_Num_Scratch_Levels = 0

IO::out_fileinfo  = "axis labels"
IO::parfile_write = "no"
```

reduce runtime to `cctk_itlast = 512` which is 1 full coarse timesteps, given 10
refinement levels)

```Python
# Finalize
Cactus::terminate           = "iteration"
Cactus::cctk_itlast         = 512
```

and add a textual description that will be shown by the test system:

```Python
!DESC "Compute center of mass of an off-center TOV star"
```

In [ ]:
%%writefile arrangements/EinsteinAnalysis/CenterOfMass/test/offcenterTOV.par
!DESC "Compute center of mass of an off-center TOV star"

# Some basic stuff
ActiveThorns = "Time MoL"
ActiveThorns = "Coordbase CartGrid3d Boundary StaticConformal"
ActiveThorns = "SymBase ADMBase TmunuBase HydroBase InitBase ADMCoupling ADMMacros"
ActiveThorns = "IOUtil"
ActiveThorns = "SpaceMask CoordGauge Constants LoopControl"
ActiveThorns = "Carpet CarpetLib CarpetReduce CarpetRegrid2"
ActiveThorns = "CarpetIOASCII"

ActiveThorns = "CenterOfMass"

# Finalize
Cactus::terminate           = "iteration"
Cactus::cctk_itlast         = 512

# grid parameters
Carpet::domain_from_coordbase = "yes"
CartGrid3D::type         = "coordbase"
CartGrid3D::domain       = "full"
CartGrid3D::avoid_origin = "no"
CoordBase::xmin = -24.0
CoordBase::ymin = -24.0
CoordBase::zmin = -24.0
CoordBase::xmax = +24.0
CoordBase::ymax = +24.0
CoordBase::zmax = +24.0
# Change these parameters to change resolution. The ?max settings above
# have to be multiples of these. 'dx' is the size of one cell in x-direction.
# Making this smaller means using higher resolution, because more points will
# be used to cover the same space.
CoordBase::dx   =   2.0
CoordBase::dy   =   2.0
CoordBase::dz   =   2.0

CarpetRegrid2::regrid_every =   0
CarpetRegrid2::num_centres  =   1
CarpetRegrid2::num_levels_1 =   2
CarpetRegrid2::radius_1[1]  = 12.0
CarpetRegrid2::position_x_1 = 3.5
CarpetRegrid2::position_y_1 = 1.5
CarpetRegrid2::position_z_1 = 2.0

CoordBase::boundary_size_x_lower        = 3
CoordBase::boundary_size_y_lower        = 3
CoordBase::boundary_size_z_lower        = 3
CoordBase::boundary_size_x_upper        = 3
CoordBase::boundary_size_y_upper        = 3
CoordBase::boundary_size_z_upper        = 3
CoordBase::boundary_shiftout_x_lower    = 0
CoordBase::boundary_shiftout_y_lower    = 0
CoordBase::boundary_shiftout_z_lower    = 0
CoordBase::boundary_shiftout_x_upper    = 0
CoordBase::boundary_shiftout_y_upper    = 0
CoordBase::boundary_shiftout_z_upper    = 0

# storage and coupling
TmunuBase::stress_energy_storage = yes
TmunuBase::stress_energy_at_RHS  = yes
TmunuBase::timelevels            =  1
TmunuBase::prolongation_type     = none


HydroBase::timelevels            = 3

ADMMacros::spatial_order = 4

SpaceMask::use_mask      = "yes"

Carpet::enable_all_storage       = no
Carpet::use_buffer_zones         = "yes"

Carpet::poison_new_timelevels    = "yes"
Carpet::check_for_poison         = "no"

Carpet::init_3_timelevels        = no
Carpet::init_fill_timelevels     = "yes"

CarpetLib::poison_new_memory = "yes"
CarpetLib::poison_value      = 114

# system specific Carpet paramters
Carpet::max_refinement_levels    = 10
driver::ghost_size               = 3
Carpet::prolongation_order_space = 3
Carpet::prolongation_order_time  = 2

# Time integration
time::dtfac = 0.25

MoL::ODE_Method             = "Euler"
MoL::MoL_Intermediate_Steps = 1
MoL::MoL_Num_Scratch_Levels = 0

# Hydro paramters

ActiveThorns = "EOS_Omni GRHydro"

HydroBase::evolution_method      = "GRHydro"

GRHydro::riemann_solver         = "Marquina"
GRHydro::GRHydro_eos_type       = "Polytype"
GRHydro::GRHydro_eos_table      = "2D_Polytrope"
GRHydro::recon_method           = "ppm"
GRHydro::GRHydro_stencil        = 3
GRHydro::bound                  = "none"
GRHydro::rho_abs_min            = 1.e-10
# Parameter controlling finite difference order of the Christoffel symbols in GRHydro
GRHydro::sources_spatial_order  = 4

# Curvature evolution parameters

ActiveThorns = "GenericFD NewRad"
ActiveThorns = "ML_BSSN ML_BSSN_Helper"
ADMBase::evolution_method        = "ML_BSSN"
ADMBase::lapse_evolution_method  = "ML_BSSN"
ADMBase::shift_evolution_method  = "ML_BSSN"
ADMBase::dtlapse_evolution_method= "ML_BSSN"
ADMBase::dtshift_evolution_method= "ML_BSSN"

ML_BSSN::timelevels = 3

ML_BSSN::harmonicN           = 1      # 1+log
ML_BSSN::harmonicF           = 2.0    # 1+log
ML_BSSN::evolveA             = 1
ML_BSSN::evolveB             = 1
# NOTE: The following parameters select geodesic slicing. This choice only enables you to evolve stationary spacetimes.
#       They will not allow you to simulate a collapsing TOV star.
ML_BSSN::ShiftGammaCoeff     = 0.0
ML_BSSN::AlphaDriver         = 0.0
ML_BSSN::BetaDriver          = 0.0
ML_BSSN::advectLapse         = 0
ML_BSSN::advectShift         = 0
ML_BSSN::MinimumLapse        = 1.0e-8

ML_BSSN::my_initial_boundary_condition = "extrapolate-gammas"
ML_BSSN::my_rhs_boundary_condition     = "NewRad"

# Some dissipation to get rid of high-frequency noise
ActiveThorns = "SphericalSurface Dissipation"
Dissipation::verbose   = "no"
Dissipation::epsdis   = 0.01
Dissipation::vars = "
        ML_BSSN::ML_log_confac
        ML_BSSN::ML_metric
        ML_BSSN::ML_curv
        ML_BSSN::ML_trace_curv
        ML_BSSN::ML_Gamma
        ML_BSSN::ML_lapse
        ML_BSSN::ML_shift
"


# init parameters
InitBase::initial_data_setup_method = "init_some_levels"

# Use TOV as initial data
ActiveThorns = "TOVSolver"

HydroBase::initial_hydro         = "tov"
ADMBase::initial_data            = "tov"
ADMBase::initial_lapse           = "tov"
ADMBase::initial_shift           = "tov"
ADMBase::initial_dtlapse         = "zero"
ADMBase::initial_dtshift         = "zero"

# Parameters for initial star
tOVSolver::TOV_Position_x[0] = 3.5
TOVSolver::TOV_Position_y[0] = 1.5
TOVSolver::TOV_Position_z[0] = 2.0

TOVSolver::TOV_Rho_Central[0] = 1.28e-3
TOVSolver::TOV_Gamma          = 2
TOVSolver::TOV_K              = 100

# Set equation of state for evolution
EOS_Omni::poly_gamma                   = 2
EOS_Omni::poly_k                       = 100
EOS_Omni::gl_gamma                     = 2
EOS_Omni::gl_k                         = 100


# I/O

# Use (create if necessary) an output directory named like the
# parameter file (minus the .par)
IO::out_dir             = ${parfile}
IO::out_fileinfo        = "axis labels"
IO::parfile_write       = "no"
        
# Write one file overall per output (variable/group)
# In production runs, comment this or set to "proc" to get one file
# per MPI process
IO::out_mode            = "onefile"

# these makes "timeseries" output via out0d_vars readable
CarpetIOASCII::compact_format = "yes"
CarpetIOASCII::one_file_per_group = yes

CarpetIOASCII::out0d_every = 1
CarpetIOASCII::out0d_vars = "CenterOfMass::center_of_mass"

## Generating test data

Simfactory's `--testsuite` option, and the underlying `sim-testsuite` make
target of Cactus, can be used to run testsuites on each of the supported
machines, but cannot be conveniently used to generate the inital set of
expected output. Instead we will run Cactus manually on the test parfile.

In [ ]:
%%bash

cd arrangements/EinsteinAnalysis/CenterOfMass/test

# remove any previous stored output
rm -rf offcenterTOV

export OMP_NUM_THREADS=2
time mpirun -n 2 ../../../../exe/cactus_sim offcenterTOV.par

tail offcenterTOV/centerofmass-center_of_mass..asc

## Testing the new test

To check that the new testsuite is fully set up, we will run it wonce.

For this notebook we use Cactus' `sim-testsuite` target and disabling
all prompts and running only the new test and verifying that is passes
using 1 MPI rank while having been produced using 2 MPI ranks.

In [ ]:
%%bash

export OMP_NUM_THREADS=1
make sim-testsuite PROMPT=no CCTK_TESTSUITE_RUN_PROCESSORS=1 CCTK_TESTSUITE_RUN_TESTS=CenterOfMass

# Extra slides

The material below is not directly used in the tutorial, it
provides some backgroun information on how Carpet traverses the
scheddule and on the interaction between `GLOBAL` and `LOCAL`
schedule functinon.

## Mesh refinement

Mesh refinement, in particular subcylcing in time, adds complications to
computing reductions (and any other "global" quantity.).

* a (`LOCAL`) scheduled function is called once per grid patch
* different refinement levels exist at different times
* `sum` like reductions have to take relative size of grid cells into account
* during `EVOL` refinement levels are updated coarse-to-fine

### Modes

Carpet introduces the concept of "modes" to add a concept of acces levels to the
Cactus schedule:

<dl>
    <dt><code>LOCAL</code></dt><dd>called once per component, has access to
    poisition dependent data on the grid</dd>
    <dt><code>LEVEL</code></dt><dd>called once per refinement level,
    <code>SYNC</code> happens here, logically boundary conditions are speficied
    in this mode, the schedule bins are traversed in this mode. </dd>
    <dt><code>GLOBAL</code></dt><dd>called once per time step, has access to
    grid scalars and grid arrays only</dd>
</dl>

`CCTK_Reduce` can be called in either `LEVEL` or `GLOBAL` mode and will return
a reduction over all values on a *single* refinement level or *all* refinement
levels.

### Interpolation in time

Since `GLOBAL` routines are called for every since timestep, and not just every
"coarse" timestep, a reduction maybe called at times that do not correspond to a
time for which a coarse refinement level exists. In the case CarpetReduce will
interpolate grid data in time to provide coarse grid data at the required time.

### Cell volumes

Naively computing the sum of values on the grid once multiple refinement levels
exist is not very useful since such an operation does not correspond to any
continuum functional of the grid variables. Carpet this weighs each contribution
by the relative cell volume, producing an approximation of the Riemann sum. Since
Carpet does now know the physical volume that a cell corresponds to, a relative
cell volume normalized by the volume of the cells on the coarsest grid is used.

### `EVOL` vs `ANALYSIS`

01234567890123456789012345678901234567890123456789012345678901234567890123456789

Refinement levels are traversed coarse to fine, so during `EVOL` Carpet takes a
step on the coarsest level first, then a step with half the stepsize on the next
finest level, etc. meaning that data one coarse levels is *ahead* in time of the
fine levels. In `ANALYSIS` the fine refinement levels have caught up with the
coarse ones, so all refinement are at the *same* time.

In `EVOL` Carpet will call `GLOBAL` scheduled functions along with `LOCAL`
routines on the coarset level (first) so that XXX.

In `ANALYSIS` Carpet will call `GLOBAL` scheduled functiosn along with `LOCAL`
routines on the finest level (last) so that a reduction over a just computed
analysis quantity is possible.

## Sketch of schedule traversal using 2 refinement levels

```Perl
schedule take_step IN EVOL
{
    LANG: C
    OPTIONS: LOCAL
} "update to next time"

schedule compute_average IN EVOL AFTER take_step
{
    LANG: C
    OPTIONS: GLOBAL
} "compute average"

schedule compute_energy IN ANALYSIS
{
    LANG: C
    OPTIONS: GLOBAL
} "compute energy density"


schedule compute_total_energy IN ANALYSIS AFTER compute_energy
{
    LANG: C
    OPTIONS: GLOBAL
} "compute total energy"

```



<table cellspacing="0" border="0">
	<colgroup span="4" width="85"></colgroup>
	<colgroup width="123"></colgroup>
	<colgroup width="143"></colgroup>
	<colgroup width="230"></colgroup>
	<colgroup span="6" width="34"></colgroup>
	<tr>
		<td height="17" align="left" bgcolor="#B3B3B3"><b>iteration</b></td>
		<td align="left" bgcolor="#B3B3B3"><b>cctk_time</b></td>
		<td align="left" bgcolor="#B3B3B3"><b>reflevel</b></td>
		<td align="left" bgcolor="#B3B3B3"><b>mode</b></td>
		<td align="left" bgcolor="#B3B3B3"><b>bin</b></td>
		<td align="left" bgcolor="#B3B3B3"><b>function</b></td>
		<td align="left" bgcolor="#B3B3B3"><b>description</b></td>
		<td colspan=3 align="left" bgcolor="#999999"><b>reflevel 0</b></td>
		<td colspan=3 align="left" bgcolor="#CCCCCC"><b>reflevel 1</b></td>
		</tr>
	<tr>
		<td height="17" align="left"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
		<td align="left">tl=0</td>
		<td align="left">tl=1</td>
		<td align="left">tl=2</td>
		<td align="left">tl=0</td>
		<td align="left">tl=1</td>
		<td align="left">tl=2</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#4C4C4C" sdval="0" sdnum="4105;"><font color="#FFFFFF">0</font></td>
		<td align="right" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#999999" sdval="0" sdnum="4105;">0</td>
		<td align="left" bgcolor="#0000FF">local</td>
		<td align="left" bgcolor="#00FF00">INITIAL</td>
		<td align="left" bgcolor="#FF00FF">intial_data</td>
		<td align="left" bgcolor="#FF00FF">set initial data</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#000000" sdval="-2" sdnum="4105;"><font color="#FFFFFF">-2</font></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#4C4C4C" sdval="0" sdnum="4105;"><font color="#FFFFFF">0</font></td>
		<td align="right" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td align="left" bgcolor="#0000FF">local</td>
		<td align="left" bgcolor="#00FF00">INITIAL</td>
		<td align="left" bgcolor="#FF00FF">intial_data</td>
		<td align="left" bgcolor="#FF00FF">set initial data</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#000000" sdval="-2" sdnum="4105;"><font color="#FFFFFF">-2</font></td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;"><font color="#FFFFFF">-0.5</font></td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
	</tr>
	<tr>
		<td style="border-top: 2px solid #000000" height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#999999" sdval="0" sdnum="4105;">0</td>
		<td style="border-top: 2px solid #000000" align="left"><br></td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#00FFFF">EVOL</td>
		<td style="border-top: 2px solid #000000" align="left"><br></td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#999999">cycle timelevels, 0-&gt;1, 1-&gt;2</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#E8E8E8"><br></td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;"><font color="#FFFFFF">-0.5</font></td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#999999" sdval="0" sdnum="4105;">0</td>
		<td align="left" bgcolor="#0000FF">local</td>
		<td align="left" bgcolor="#00FFFF">EVOL</td>
		<td align="left" bgcolor="#FF950E">take_step</td>
		<td align="left" bgcolor="#FF950E">update level to time 1.00</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;"><font color="#FFFFFF">-0.5</font></td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#FFFF00">global</td>
		<td align="left" bgcolor="#00FFFF">EVOL</td>
		<td align="left" bgcolor="#0000FF">compute_average</td>
		<td align="left" bgcolor="#0000FF">compute average</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;"><font color="#FFFFFF">-0.5</font></td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#B3B3B3" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#00FFFF">EVOL</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#999999">cycle timelevels, 0-&gt;1, 1-&gt;2</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="left" bgcolor="#BABABA"><br></td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;">-0.5</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#B3B3B3" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td align="left" bgcolor="#0000FF">local</td>
		<td align="left" bgcolor="#00FFFF">EVOL</td>
		<td align="left" bgcolor="#FF950E">take_step</td>
		<td align="left" bgcolor="#FF950E">update level to time 0.50</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;">-0.5</td>
	</tr>
	<tr>
		<td style="border-bottom: 2px solid #000000" height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="left"><br></td>
		<td style="border-bottom: 2px solid #000000" align="left" bgcolor="#FFFF00">global</td>
		<td style="border-bottom: 2px solid #000000" align="left" bgcolor="#00FFFF">EVOL</td>
		<td style="border-bottom: 2px solid #000000" align="left" bgcolor="#0000FF">compute_average</td>
		<td style="border-bottom: 2px solid #000000" align="left" bgcolor="#0000FF">compute average</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;">-0.5</td>
	</tr>
	<tr>
		<td style="border-top: 2px solid #000000" height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#B3B3B3" sdval="0.5" sdnum="4105;">0.5</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#0000FF">local</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#FFD320">ANALYSIS</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#FFFF00">compute_energy</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#FFFF00">compute eergy density at time 0.50</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;">-0.5</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#B3B3B3" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#FFFF00">global</td>
		<td align="left" bgcolor="#FFD320">ANALYSIS</td>
		<td align="left" bgcolor="#23FF23">compute_total_energy</td>
		<td align="left" bgcolor="#23FF23">compute total energy</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;">-0.5</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#B3B3B3" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td align="left" bgcolor="#FF9966">level</td>
		<td align="left" bgcolor="#FF00FF">OutputGH</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#FF00FF">output level 1 data at t=0.50</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;">-0.5</td>
	</tr>
	<tr>
		<td style="border-bottom: 2px solid #000000" height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#B3B3B3" sdval="0.5" sdnum="4105;">0.5</td>
		<td style="border-bottom: 2px solid #000000" align="left"><br></td>
		<td style="border-bottom: 2px solid #000000" align="left"><br></td>
		<td style="border-bottom: 2px solid #000000" align="left" bgcolor="#FF00FF">OutputGH</td>
		<td style="border-bottom: 2px solid #000000" align="left"><br></td>
		<td style="border-bottom: 2px solid #000000" align="left" bgcolor="#00FFFF">output scalars at t=0.50</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;">-0.5</td>
	</tr>
	<tr>
		<td style="border-top: 2px solid #000000" height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="left"><br></td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#00FFFF">EVOL</td>
		<td style="border-top: 2px solid #000000" align="left"><br></td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#999999">cycle timelevels, 0-&gt;1, 1-&gt;2</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#E8E8E8"><br></td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td align="left" bgcolor="#0000FF">local</td>
		<td align="left" bgcolor="#00FFFF">EVOL</td>
		<td align="left" bgcolor="#FF950E">take_step</td>
		<td align="left" bgcolor="#FF950E">update level to time 1.00</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#FFFF00">global</td>
		<td align="left" bgcolor="#00FFFF">EVOL</td>
		<td align="left" bgcolor="#0000FF">compute_average</td>
		<td align="left" bgcolor="#0000FF">compute average</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td style="border-bottom: 2px solid #000000" height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="left"><br></td>
		<td style="border-bottom: 2px solid #000000" align="left" bgcolor="#00FFFF">EVOL</td>
		<td style="border-bottom: 2px solid #000000" align="left"><br></td>
		<td style="border-bottom: 2px solid #000000" align="left" bgcolor="#CCCCCC">restrict fine to coarse</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td style="border-top: 2px solid #000000" height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#999999" sdval="0" sdnum="4105;">0</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#0000FF">local</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#FFD320">ANALYSIS</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#FFFF00">compute_energy</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#FFFF00">compute eergy density at time 1.00</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#999999" sdval="0" sdnum="4105;">0</td>
		<td align="left" bgcolor="#FF9966">level</td>
		<td align="left" bgcolor="#FF00FF">OutputGH</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#FF00FF">output level 0 data at t=1.00</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td align="left" bgcolor="#0000FF">local</td>
		<td align="left" bgcolor="#FFD320">ANALYSIS</td>
		<td align="left" bgcolor="#FFFF00">compute_energy</td>
		<td align="left" bgcolor="#FFFF00">compute eergy density at time 1.00</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#FFFF00">global</td>
		<td align="left" bgcolor="#FFD320">ANALYSIS</td>
		<td align="left" bgcolor="#23FF23">compute_total_energy</td>
		<td align="left" bgcolor="#23FF23">compute total energy</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td align="left" bgcolor="#FF9966">level</td>
		<td align="left" bgcolor="#FF00FF">OutputGH</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#FF00FF">output level 1 data at t=1.00</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#FFFF00">global</td>
		<td align="left" bgcolor="#FF00FF">OutputGH</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#00FFFF">output scalars at t=1.00</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
</table>
